In [1]:
import pandas as pd
import numpy as np


In [42]:
# import and combine scraped data so far
df0 = pd.read_csv("solar_df.csv")
df1 = pd.read_csv("solar_df_jan23_1.csv") 
df2 = pd.read_csv("solar_df_jan23_2.csv")
df3 = pd.read_csv("solar_df_jan23_3.csv")
df4 = pd.read_csv("solar_df_jan23_4.csv")

dfs = [df0, df1, df2, df3, df4] 

df = pd.concat(dfs)

In [43]:
len(df)

10246

In [44]:
for x in dfs:
    print(len(x))

2360
1927
1949
2005
2005


# Efficiency

- need to transform into numeric type
- some values are given as point, other as intervals -- need to unify
- where min-max is given, mean will be used; point values will be used as is

In [46]:
#create helper column to flag interval vs. point value
def single_flag(value):
    if "~" in value:
        return("interval")
    else:
        return("point")

In [47]:
df['efficiency_value'] = df['efficiency'].map(lambda x: single_flag(x))

In [48]:
df_point = df[df['efficiency_value'] == "point"]

In [49]:
pd.options.mode.chained_assignment = None

In [50]:
df_point['efficiency_percent'] = df_point['efficiency'].str.replace("%", "")

In [51]:
df_point['efficiency_percent'] = np.where(df_point['efficiency_percent'].str.contains("-"),
                                             "NaN",
                                             df_point['efficiency_percent'])

In [52]:
df_point['efficiency_percent'] = df_point['efficiency_percent'].astype(float)

In [53]:
df_interval = df[df['efficiency_value'] == "interval"]

In [54]:
df_interval['efficiency_min'] =  df_interval["efficiency"].map(lambda x: x.split("~")[0])

In [55]:
df_interval['efficiency_max'] =  df_interval["efficiency"].map(lambda x: x.split("~")[1])

In [56]:
df_interval["efficiency_max"] = df_interval['efficiency_max'].map(lambda x: x.replace("%", ""))

In [57]:
df_interval['efficiency_min'] = df_interval['efficiency_min'].astype(float)
df_interval['efficiency_max'] = df_interval['efficiency_max'].astype(float)

In [58]:
df_interval['efficiency_percent'] = (df_interval['efficiency_min'] + df_interval['efficiency_max'])/2

In [59]:
df = pd.concat([df_point, df_interval], join = "inner")

# Price 
- some entries for price seem to be wrong: they have to be divided by 100 to make sense
- there are 2 values at around EURO 40 -- we cannot know what the intention was here
- note that the "expensive" ones which cost above 2-3 EUR but below 5 are extremely light, whereas the outliers (>100EURO) weigh the same as the regular priced panels (above 10-30kg)

In [60]:
df['price_euro'] = df['price'].map(lambda x: x.replace("\u20ac", "")).astype(float)

In [61]:
# divide the prices above 100 by 100
df['price_euro'] = np.where(df['price_euro'] > 100,
                              df['price_euro']/100,
                              df['price_euro'])


In [62]:
# transform the prices below 100 but above 10 into NA (2 remaining)
df['price_euro'] = np.where(df['price_euro'] > 40,
                              float('nan'),
                              df['price_euro'])


# Brand

- Create "other" category for brands that have fewer than 10 datapoints

In [63]:
df['brand_frequency'] = df.groupby('brand')['brand'].transform("count")

In [64]:
df.sort_values(by = 'brand_frequency').iloc[240:250]

,Unnamed: 0,series,brand,panel_type,efficiency,power,weight,region,dimension,price,efficiency_value,efficiency_percent,price_euro,brand_frequency
6,6,(Shingled) ECO-480-500M-72SB,ECO DELTA,Monocrystalline,20.5 ~ 21.33 %,480 ~ 500 Wp,25 kg,China,2056x1140x35 mm,€0.320,interval,20.915,0.320,8
679,679,WP365-380/G6-120H,Wattpower,PERC,20 ~ 20.9 %,365 ~ 380 Wp,19.5 kg,Germany,1755x1038x35 mm,€0.283,interval,20.450,0.283,8
684,684,AstroSemi CHSM72M-HC 525W-545W,Astronergy,Monocrystalline,20.5 ~ 21.3 %,525 ~ 545 Wp,27.2 kg,Spain,2256x1133x35 mm,€0.208,interval,20.900,0.208,8
116,116,(Shingled) ECO-395-415M-60SB,ECO DELTA,Monocrystalline,20.2 ~ 21.2 %,395 ~ 415 Wp,21 kg,China,1719x1140x30 mm,€0.320,interval,20.700,0.320,8
412,412,GPMDG-540W(144),G&P Sun Energy Technology,Monocrystalline,20.84 %,540 Wp,31.6 kg,China,2278x1134x35 mm,€159,point,20.840,1.590,8
278,278,NRS-FLX-75-510W,NRSun,Thin film (CIS),17 %,125 ~ 510 Wp,--,Belgium,--,€166,point,17.000,1.660,8
309,309,Mono 6,Greco Solar,Monocrystalline,14.8 ~ 16 %,240 ~ 260 Wp,20 kg,China,1640x992x40 mm,€0.223,interval,15.400,0.223,8
314,314,ASTRO 5s CHSM54M-HC 400~415W,Astronergy,Monocrystalline,20.5 ~ 21.3 %,400 ~ 415 Wp,21.5 kg,Spain,1708x1133x30 mm,€0.251,interval,20.900,0.251,8
328,328,XSM-435-455M-SS,Newsunmi New Energy,Monocrystalline,20.01 ~ 20.93 %,435 ~ 455 Wp,24.5 kg,China,2102x1040x35 mm,€10.0,interval,20.470,10.000,8
684,684,AstroSemi CHSM72M-HC 525W-545W,Astronergy,Monocrystalline,20.5 ~ 21.3 %,525 ~ 545 Wp,27.2 kg,Spain,2256x1133x35 mm,€0.208,interval,20.900,0.208,8


In [65]:
brand_count = df['brand'].value_counts()

In [66]:
other_brands = list(brand_count[brand_count < 10].index)

In [67]:
df['brand_bucketed'] = df['brand'].replace(other_brands, 'Other')

# Panel Type

- like for brand: create "Other" category for those with fewer than 10 datapoints

In [68]:
panel_count = df['panel_type'].value_counts()

In [69]:
other_panel = list(panel_count[panel_count < 10].index)

In [70]:
df['panel_type_bucketed'] = df['panel_type'].replace(other_panel, "Other")

# Weight

In [71]:
df['weight_kg'] = np.where(df['weight'].str.contains("-"),
                           "Nan",
                           df['weight'].map(lambda x: x.split("kg")[0])
                              )

In [72]:
df['weight_kg'] = df['weight_kg'].astype(float)

# Region

In [73]:
df['region'] = df['region'].map(lambda x: x.strip())

In [74]:
region_count = df['region'].value_counts()

In [75]:
other_region = list(region_count[region_count < 10].index)

In [76]:
df['region_bucketed'] = df['region'].replace(other_region, "Other")

# Drop unused columns

In [77]:
df.columns

Index(['Unnamed: 0', 'series', 'brand', 'panel_type', 'efficiency', 'power',
       'weight', 'region', 'dimension', 'price', 'efficiency_value',
       'efficiency_percent', 'price_euro', 'brand_frequency', 'brand_bucketed',
       'panel_type_bucketed', 'weight_kg', 'region_bucketed'],
      dtype='object')

In [78]:
df.drop(['Unnamed: 0', 'brand', 'panel_type',  'efficiency', 'power', 
        'weight', 'region', 'dimension', 'price', 'efficiency_value', 
        'brand_frequency'], 
        axis=1,
        inplace=True)

# Remove Duplicates

In [80]:
print(len(df))
print(df.duplicated().sum())

10246
6978


In [81]:
df.drop_duplicates(inplace=True)

# Save

In [82]:
# save to csv
df.to_csv('df_clean.csv')

In [83]:
df_clean = pd.read_csv("df_clean.csv")

In [84]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3268 entries, 0 to 3267
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           3268 non-null   int64  
 1   series               3268 non-null   object 
 2   efficiency_percent   3116 non-null   float64
 3   price_euro           3267 non-null   float64
 4   brand_bucketed       3268 non-null   object 
 5   panel_type_bucketed  3268 non-null   object 
 6   weight_kg            3116 non-null   float64
 7   region_bucketed      3268 non-null   object 
dtypes: float64(3), int64(1), object(4)
memory usage: 204.4+ KB
